In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
from selectolax.parser import HTMLParser
import html5lib

In [2]:
df = pd.read_csv('player_stats.csv')

In [3]:
df.head()

,Unnamed: 0,Name,Position,StatDict
0,Lukáš Hrádecký,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_keeper_dom_lg': '<table class=""stats_...",NaN
1,Granit Xhaka,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat...",NaN
2,Álex Grimaldo,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat...",NaN
3,Jonathan Tah,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat...",NaN
4,Jeremie Frimpong,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat...",NaN


In [4]:
df.shape

(3319, 4)

In [5]:
df.drop(columns=['StatDict'], inplace=True)

In [6]:
df.rename(columns={'Position':'StatDict', 'Name':'Position', 'Unnamed: 0':'Name'}, inplace=True)

In [7]:
df.head()

,Name,Position,StatDict
0,Lukáš Hrádecký,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_keeper_dom_lg': '<table class=""stats_..."
1,Granit Xhaka,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat..."
2,Álex Grimaldo,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat..."
3,Jonathan Tah,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat..."
4,Jeremie Frimpong,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","{'#stats_standard_dom_lg': '<table class=""stat..."


In [8]:
pos_html = df['Position'][0]

In [9]:
stats_html = eval(df['StatDict'][0])

In [10]:
def extract_player_info(pos_html):
    info = []
    for node in HTMLParser(pos_html).css('p'):
        name = node.text()
        info.append(name.strip())
        
    attrs = []
    for val in info:
        val = re.sub('▪| |\xa0|\n','', val)
        attrs.append(val)
        
    data = "\n".join(attrs)
    
    position_pattern = r'Position:\s*([A-Z\-\/]+)(?:\s*\(.*\))?'  # Capture main position before parentheses
    footed_pattern = r'Footed:\s*(\w+)'  # Capture footed (left/right)
    height_pattern = r'(\d{3})cm'  # Capture height in cm
    weight_pattern = r'(\d{2,3})kg'  # Capture weight in kg
    age_pattern = r'Age:(\d+)'  # Capture age in years
    national_team_pattern = r'National Team:\s*([A-Za-z\s]+)'  # Capture national team
    club_pattern = r'Club:\s*([A-Za-z\s]+)'  # Capture club

    # Function to extract data
    def extract_data(pattern, text):
        match = re.search(pattern, text)
        return match.group(1) if match else np.nan

    # Extracting fields
    position = extract_data(position_pattern, data)
    footed = extract_data(footed_pattern, data)
    height = extract_data(height_pattern, data)
    weight = extract_data(weight_pattern, data)
    age = extract_data(age_pattern, data)
    national_team = extract_data(national_team_pattern, data)
    club = extract_data(club_pattern, data)

    return {
        "positon"         :position,
        "footed"          :footed,
        "height"          :height,
        "weight"          :weight,
        "age"             :age,
        "national_team"   :national_team,
        "current_club"    :club
    }

In [11]:
peronsal = extract_player_info(pos_html)

In [12]:
def process_stats(stats_html):
    table_names = dict()
    for table in stats_html.keys():
        for node in HTMLParser(stats_html[table]).css('caption'):
            text = node.text().split(':')
            table_names[table] = text[0]
    final_dfs = []
    for table in table_names.keys():
        stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
        new_df = pd.DataFrame()
        for col in stats_df.columns:
            if('Unnamed' in col[0]):
                new_df[str(col[1])] = stats_df[col[0]][col[1]]
            else:
                new_df[str(col[0]) + '_' + str(col[1])] = stats_df[col[0]][col[1]]

        stats_df = pd.DataFrame(columns=new_df.columns)
        df_idx = 0
        for idx, row in new_df.iterrows():
            if((row['Season'] is np.nan) or (row['Squad'] is np.nan)):
                continue
            if(('Seasons' in row['Season']) or ('Clubs' in row['Squad'])):
                break
            stats_df.loc[df_idx] = row
            seasons = row['Season'].split('-')
            stats_df.at[df_idx, 'for_join'] = str(int(seasons[0])) + '-' + str(int(seasons[1])) + '-' + str(df_idx)
            df_idx += 1
        final_dfs.append(stats_df)
    statistics = pd.DataFrame()
    for i in range(len(final_dfs)):
        if i == 0:
            statistics = final_dfs[i].copy()
            continue
        result = pd.concat([statistics, final_dfs[i]], axis=1, join='inner')
        result = result.loc[:,~result.columns.duplicated()]
        statistics = result.copy()
    statistics.drop(columns=['for_join'], inplace=True)
    return statistics

In [13]:
stats_df = process_stats(stats_html)

C:\Users\shour\AppData\Local\Temp\ipykernel_18676\596878951.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
C:\Users\shour\AppData\Local\Temp\ipykernel_18676\596878951.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
C:\Users\shour\AppData\Local\Temp\ipykernel_18676\596878951.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
C:\Users\shour\AppData\Local

In [14]:
stats_df.head()

,Season,Age,Squad,Country,Comp,LgRank,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Performance_Crs,Performance_Int,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,2009-2010,19,Esbjerg,dk DEN,1. Danish Superliga,4th,5,3,356,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-2011,20,Esbjerg,dk DEN,1. Danish Superliga,12th,13,13,1170,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-2013,22,Esbjerg,dk DEN,1. Danish Superliga,4th,33,33,2970,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-2014,23,Brøndby,dk DEN,1. Danish Superliga,4th,33,33,2970,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-2015,24,Brøndby,dk DEN,1. Danish Superliga,3rd,33,33,2970,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
stats_df

,Season,Age,Squad,Country,Comp,LgRank,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Performance_Crs,Performance_Int,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,2009-2010,19,Esbjerg,dk DEN,1. Danish Superliga,4th,5,3,356,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-2011,20,Esbjerg,dk DEN,1. Danish Superliga,12th,13,13,1170,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-2013,22,Esbjerg,dk DEN,1. Danish Superliga,4th,33,33,2970,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-2014,23,Brøndby,dk DEN,1. Danish Superliga,4th,33,33,2970,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-2015,24,Brøndby,dk DEN,1. Danish Superliga,3rd,33,33,2970,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-2016,25,Brøndby,dk DEN,1. Danish Superliga,4th,3,3,270,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-2016,25,Eint Frankfurt,de GER,1. Bundesliga,16th,34,34,3060,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016-2017,26,Eint Frankfurt,de GER,1. Bundesliga,11th,33,33,2883,32.0,...,0,3,0,0,0,0,NaN,NaN,NaN,NaN
8,2017-2018,27,Eint Frankfurt,de GER,1. Bundesliga,8th,34,34,3060,34.0,...,0,0,0,0,0,0,74,7,0,100.0
9,2018-2019,28,Leverkusen,de GER,1. Bundesliga,4th,32,32,2880,32.0,...,0,1,0,0,0,0,64,7,0,100.0


In [16]:
final_df = pd.read_csv('statistics.csv')

In [17]:
final_df.head()

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club,Playing Time_MP
0,2010-2011,17,Basel,ch SUI,1. Super Lg,1st,20.0,11.0,1136.0,12.6,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
1,2011-2012,18,Basel,ch SUI,1. Super Lg,1st,24.0,18.0,1595.0,17.7,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
2,2012-2013,19,Gladbach,de GER,1. Bundesliga,8th,22.0,15.0,1477.0,16.4,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
3,2013-2014,20,Gladbach,de GER,1. Bundesliga,6th,28.0,23.0,2023.0,22.5,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
4,2014-2015,21,Gladbach,de GER,1. Bundesliga,3rd,30.0,30.0,2637.0,29.3,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN


In [18]:
df = pd.read_csv('statistics.csv')

In [19]:
df.head()

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club,Playing Time_MP
0,2010-2011,17,Basel,ch SUI,1. Super Lg,1st,20.0,11.0,1136.0,12.6,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
1,2011-2012,18,Basel,ch SUI,1. Super Lg,1st,24.0,18.0,1595.0,17.7,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
2,2012-2013,19,Gladbach,de GER,1. Bundesliga,8th,22.0,15.0,1477.0,16.4,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
3,2013-2014,20,Gladbach,de GER,1. Bundesliga,6th,28.0,23.0,2023.0,22.5,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
4,2014-2015,21,Gladbach,de GER,1. Bundesliga,3rd,30.0,30.0,2637.0,29.3,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN


In [20]:
df

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club,Playing Time_MP
0,2010-2011,17,Basel,ch SUI,1. Super Lg,1st,20.0,11.0,1136.0,12.6,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
1,2011-2012,18,Basel,ch SUI,1. Super Lg,1st,24.0,18.0,1595.0,17.7,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
2,2012-2013,19,Gladbach,de GER,1. Bundesliga,8th,22.0,15.0,1477.0,16.4,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
3,2013-2014,20,Gladbach,de GER,1. Bundesliga,6th,28.0,23.0,2023.0,22.5,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
4,2014-2015,21,Gladbach,de GER,1. Bundesliga,3rd,30.0,30.0,2637.0,29.3,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24188,2020-2021,18,Sheffield Utd,eng ENG,1. Premier League,20th,1.0,0.0,1.0,0.0,...,0.0,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN
24189,2021-2022,19,Sheffield Utd,eng ENG,2. Championship,5th,1.0,1.0,66.0,0.7,...,NaN,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN
24190,2021-2022,19,Beerschot,be BEL,1. First Division A,18th,1.0,0.0,8.0,0.1,...,NaN,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN
24191,2022-2023,20,Rochdale,eng ENG,4. League Two,24th,29.0,20.0,1821.0,20.2,...,50.0,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN


In [21]:
df['position'].value_counts()

position
DF           5261
MF           4300
FW-MF        3873
DF-MF        3840
FWF          2142
FW-MFF       1597
MFF           863
FW            804
DF-FW-MF      745
DFF           338
DF-MFF        217
DF-FW-MFF     138
DF-FW          49
DF-FWF         24
Name: count, dtype: int64

In [22]:
gk_df = pd.read_csv('gk_statistics.csv')

In [23]:
gk_df['position'].value_counts()

position
GKF    2467
GK      211
Name: count, dtype: int64

In [24]:
gk_df[gk_df['position'] == 'Right']

,Season,Age,Squad,Country,Comp,LgRank,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Lost,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club


In [26]:
df = pd.read_csv('statistics.csv')
gk_df = pd.read_csv('gk_statistics.csv')

In [27]:
df.shape

(24193, 186)

In [29]:
gk_df['position'].value_counts()

position
GKF    2467
GK      211
Name: count, dtype: int64

In [30]:
gk_df.shape

(2678, 211)